In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
 
# Load the ChEMBL dataset
df = pd.read_csv('/content/chembl_dataset_7956_smiles.csv')
 
# Define the input features and output feature
x_features = 4
y_feature = 'logP'
 
# Extract the input features from the SMILES strings
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    features = [
        Descriptors.MolWt(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        mol.GetRingInfo().NumRings()
    ]
    return np.array(features)

def extract_output_feature(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    logp = Chem.rdMolDescriptors.CalcCrippenDescriptors(mol)[0]
    return logp
 
df['features'] = df['smiles'].apply(extract_features)

# Remove rows with missing features
df.dropna(subset=['features'], inplace=True)

# adding output:
df[y_feature] = df['smiles'].apply(extract_output_feature)

df.dropna(subset=[y_feature], inplace=True)
# print(df)
 
# num_atoms = mol.GetNumAtoms()  # number of atoms

# Extract the output feature as the target variable
y = df[y_feature]
 
# Convert the input features to a NumPy array
x = np.stack(df['features'])
 
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Define the architecture of the neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(x_features,), activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
 
# Compile the model
model.compile(loss='mse', optimizer='adam')
 
# Train the model
model.fit(x_train, y_train, epochs=50, validation_split=0.2)
 
# Evaluate the model on the testing set
loss = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.2f}')
 
# Make a prediction using the trained model
x_new = np.array([[300.0, 50.0, 4, 1]])  # example input features
y_new = model.predict(x_new)
print(y_new)
print(f'Prediction: {y_new[0][0]:.2f}')


Epoch 1/50
160/160 [==============================] - 2s 5ms/step - loss: 7096.0933 - val_loss: 51.8271
Epoch 2/50
160/160 [==============================] - 1s 4ms/step - loss: 30.0189 - val_loss: 23.6429
Epoch 3/50
160/160 [==============================] - 0s 3ms/step - loss: 14.3159 - val_loss: 10.3757
Epoch 4/50
160/160 [==============================] - 0s 2ms/step - loss: 7.9858 - val_loss: 6.1627
Epoch 5/50
160/160 [==============================] - 0s 3ms/step - loss: 6.0746 - val_loss: 4.7679
Epoch 6/50
160/160 [==============================] - 0s 2ms/step - loss: 5.2816 - val_loss: 4.0406
Epoch 7/50
160/160 [==============================] - 0s 2ms/step - loss: 4.7699 - val_loss: 3.6314
Epoch 8/50
160/160 [==============================] - 0s 3ms/step - loss: 4.3521 - val_loss: 3.2435
Epoch 9/50
160/160 [==============================] - 0s 3ms/step - loss: 3.9623 - val_loss: 2.8772
Epoch 10/50
160/160 [==============================] - 0s 3ms/step - loss: 3.6030 - val_loss

1/1 [==============================] - 0s 88ms/step
[[2.634779]]
Prediction: 2.63


In [32]:
print(x_train)


[[439.559  67.87    5.      4.   ]
 [471.535 127.25    6.      4.   ]
 [630.69  130.12    5.      8.   ]
 ...
 [410.911 103.56    5.      3.   ]
 [377.404  83.32    6.      4.   ]
 [251.307  52.33    0.      3.   ]]


In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model

# # Load Chembl dataset
# df = pd.read_csv('/content/chembl_dataset_7956_smiles.csv')

# # Extract features from SMILES representation
# mol_list = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]
# # df['num_atoms'] = [mol.GetNumAtoms() for mol in mol_list]
# df['mol_weight'] = [Descriptors.MolWt(mol) for mol in mol_list]
# df['num_rings'] = [mol.GetRingInfo().NumRings() for mol in mol_list]
# # df['num_hbd'] = [Chem.rdMolDescriptors.CalcNumLipinskiHBD(mol) for mol in mol_list]
# # df['num_hba'] = [Chem.rdMolDescriptors.CalcNumLipinskiHBA(mol) for mol in mol_list]
# df['rotable_bonds'] = [Descriptors.NumRotatableBonds(mol) for mol in mol_list]
# df['TPSA'] = [Descriptors.TPSA(mol) for mol in mol_list]
# df['logP'] = [Chem.rdMolDescriptors.CalcCrippenDescriptors(mol)[0] for mol in mol_list]

# # Define input and output features
# input_features = ['TPSA', 'mol_weight', 'num_rings', 'rotable_bonds']
# output_features = ['logP']

# # Split dataset into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(df[input_features], df[output_features], test_size=0.2, random_state=42)

# # Standardize input features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Load the ChEMBL dataset
df = pd.read_csv('/content/chembl_dataset_7956_smiles.csv')
 
# Define the input features and output feature
x_features = 4
y_feature = 'logP'
 
# Extract the input features from the SMILES strings
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    features = [
        Descriptors.MolWt(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        mol.GetRingInfo().NumRings()
    ]
    return np.array(features)

def extract_output_feature(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    logp = Chem.rdMolDescriptors.CalcCrippenDescriptors(mol)[0]
    return logp
 
df['features'] = df['smiles'].apply(extract_features)

# Remove rows with missing features
df.dropna(subset=['features'], inplace=True)

# adding output:
df[y_feature] = df['smiles'].apply(extract_output_feature)

df.dropna(subset=[y_feature], inplace=True)
# print(df)
 
# num_atoms = mol.GetNumAtoms()  # number of atoms

# Extract the output feature as the target variable
y = df[y_feature]
 
# Convert the input features to a NumPy array
x = np.stack(df['features'])
 
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Create neural network model
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(256)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    out = Dense(1, activation="linear")(x)
    model = Model(inputs=inp, outputs=out)
    return model

# Train neural network model
model = create_nn_model(X_train.shape[1])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_test, y_test))

# Evaluate neural network model
loss, mae = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.2f}')


Epoch 1/50
50/50 [==============================] - 10s 98ms/step - loss: 5.1826 - mae: 1.6544 - val_loss: 6.8490 - val_mae: 2.1820
Epoch 2/50
50/50 [==============================] - 5s 100ms/step - loss: 2.7837 - mae: 1.1275 - val_loss: 3.2631 - val_mae: 1.3441
Epoch 3/50
50/50 [==============================] - 5s 106ms/step - loss: 2.3260 - mae: 1.0816 - val_loss: 3.1285 - val_mae: 1.3416
Epoch 4/50
50/50 [==============================] - 5s 92ms/step - loss: 2.3108 - mae: 1.0604 - val_loss: 2.4700 - val_mae: 1.1432
Epoch 5/50
50/50 [==============================] - 6s 115ms/step - loss: 2.3557 - mae: 1.0622 - val_loss: 2.2009 - val_mae: 1.0709
Epoch 6/50
50/50 [==============================] - 5s 91ms/step - loss: 2.0451 - mae: 1.0345 - val_loss: 2.1835 - val_mae: 1.0712
Epoch 7/50
50/50 [==============================] - 5s 91ms/step - loss: 1.8934 - mae: 1.0072 - val_loss: 1.5883 - val_mae: 0.9167
Epoch 8/50
50/50 [==============================] - 6s 116ms/step - loss: 1.818

In [52]:
x_new = np.array(scaler.transform([[300.0, 50.0, 4, 1]]))  # example input features
y_new = model.predict(x_new)
print(y_new)
print(f'Prediction: {y_new[0][0]:.2f}')

1/1 [==============================] - 0s 53ms/step
[[3.6608336]]
Prediction: 3.66


In [39]:
print(X_train)

[[-0.39915238  0.02333863 -0.30487919  0.34093125]
 [ 0.15608182  0.74650347  0.05090171 -0.4041534 ]
 [-0.05591928  2.66987773 -1.016441   -2.63940735]
 ...
 [-0.3415546  -0.03473164  0.05090171 -0.4041534 ]
 [ 0.44520479  0.28166994 -0.12698874  1.83110056]
 [ 0.08031626 -0.36076637 -0.30487919  1.0860159 ]]
